### 회귀 (100점)
1) 데이터 선정(20점):
- 어떠한 데이터라도 상관 없음
(캐글 데이터, 인터넷에서 구한 데이터, 직접 수집한 데 이터 등)
https://www.kaggle.com/datasets
- 데이터를 선정한 이유 서술

2) 데이터 전처리(25점): 방법, 이유, 파이썬 코드

3) 회귀 및 파라미터 최적화(30점): 방법, 이유, 파이 썬 코드

4) 결과 및 분석, 고찰(25점)

선형 회귀
폴리노미어
트리

* age: age of primary beneficiary 
* sex: insurance contractor gender, female, male 
* bmi: Body mass index, providing an understanding of body, weights that are relatively high or low relative to height, objective index of body weight (kg / m ^ 2) using the ratio of height to weight, ideally 18.5 to 24.9 
* children: Number of children covered by health insurance / Number of dependents
* smoker: Smoking
* region: the beneficiary's residential area in the US, northeast, southeast, southwest, northwest.
* charges: Individual medical costs billed by health insurance

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler, MinMaxScaler 
from sklearn.ensemble import RandomForestRegressor 
from sklearn import metrics
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import AdaBoostRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor

In [ ]:
df = pd.read_csv("insurance.csv")
df.head()

In [ ]:
df.shape

### Exploratory Data Analysis

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
df.isnull().sum()

### Encoding

In [ ]:
df.sex.value_counts()

In [ ]:
# Encoding for Sex
value1 = {'male': 1, 'female': 0}
df['sex'].replace(value1,inplace = True)
df.head()

In [ ]:
df.smoker.value_counts()

In [ ]:
value2 = {"yes": 1,"no": 0}
df['smoker'].replace(value2,inplace= True)
df.head()

In [ ]:
df.region.value_counts()

In [ ]:
value3 = {"northeast": 0,"northwest": 1, 
          "southwest": 3, "southeast": 4}
df['region'].replace(value3,inplace= True)
df.head()

In [ ]:
df.info()

In [ ]:
sns.heatmap(df.corr(),annot = True)

In [ ]:
sns.catplot(x='smoker',kind = 'count',data=df)

In [ ]:
df.smoker.value_counts()

In [ ]:
f= plt.figure(figsize=(12,5))

ax=f.add_subplot(121)
sns.distplot(df[(df.smoker == 1)]["charges"],color='c',ax=ax)
ax.set_title('Distribution of charges for smokers')

ax=f.add_subplot(122)
sns.distplot(df[(df.smoker == 0)]['charges'],color='b',ax=ax)
ax.set_title('Distribution of charges for non-smokers')

In [ ]:
mask = df['smoker'].isin([1])
df = df[~mask]
df =df.drop('smoker',axis=1)
df.head()

In [ ]:
df.reset_index()

In [ ]:
df.info()

In [ ]:
df1 = df

In [ ]:
plt.figure(figsize=(12,5))
plt.title("Distribution of charges")
ax = sns.distplot(df["charges"], color = 'm')

In [ ]:
df['charges'].hist(bins='auto')

In [ ]:
y_log = np.log1p(df['charges'])
y_log.hist(bins='auto')

In [ ]:
plt.figure(figsize=(12,5))
plt.title("Distribution of age")
ax = sns.distplot(df["age"], color = 'm')

In [ ]:
plt.figure(figsize=(12,5))
plt.title("Distribution of bmi")
ax = sns.distplot(df["bmi"], color = 'm')

In [ ]:
fig, axs = plt.subplots(2,3,sharey=True)
df.plot(kind='scatter', x='age', y='charges', ax=axs[0][0], figsize=[16,8])
df.plot(kind='scatter', x='sex', y='charges', ax=axs[0][1])
df.plot(kind='scatter', x='bmi', y='charges', ax=axs[0][2])
df.plot(kind='scatter', x='children', y='charges', ax=axs[1][0])
df.plot(kind='scatter', x='region', y='charges', ax=axs[1][1])

### 단순 선형 회귀

#### 경사하강법

In [ ]:
# 경사하강법 - 행렬 풀이
X = df.age
y = y_log

def get_cost(y, y_pred):
    N = len(y)
    cost = np.sum(np.square(y - y_pred)) / N #손실 함수 or 비용 함수
    return cost

def get_weight_updates(w1, w0, X, y, learning_rate=0.0001):
    N = len(y)
    y_pred = np.dot(X, w1) + w0
    print(get_cost(y, y_pred))
    diff = y_pred - y
    ones = np.ones((N, 1))
    w1_update = learning_rate * 2 * np.dot(X.T, diff) / N
    w0_update = learning_rate * 2 * np.dot(ones.T, diff) / N
    return w1_update, w0_update

def gradient_descent_steps(X, y, iters=10000):
    w0 = 0
    w1 = 0
    for _ in range(iters):
        w1_update, w0_update = get_weight_updates(w1, w0, X ,y)
        w1 = w1 - w1_update
        w0 = w0 - w0_update
    return w1, w0

w1, w0 = gradient_descent_steps(X, y, 1000)
print(w1, w0)

In [ ]:
y_pred1 = w1 * X + w0
plt.scatter(X, y)
plt.plot(X, y_pred1, c='red')

In [ ]:
# 경사하강법 - 편미분 방정식 풀이
X = df.age
y = y_log

def get_cost(y, y_pred):
    N = len(y)
    cost = np.sum(np.square(y-y_pred))/N
    return cost

def get_weight_updates(w1, w0, X, y):
    N = len(y)
    y_pred =np.sum(X*y)+w0
    print(get_cost(y, y_pred))
    xy_bar = np.sum(X*y)/N
    y_bar = np.sum(y)/N
    x_bar = np.sum(X)/N
    xx_bar =np.sum(X*X)/N
    return xy_bar, y_bar ,x_bar, xx_bar 

def gradient_descent_steps(X, y, iters=10000):
    w0 = 0
    w1 = 0
    for _ in range(iters):
        xy_bar, y_bar, x_bar, xx_bar = get_weight_updates(w1, w0, X, y)
        w1 =(xy_bar - x_bar*y_bar) / (xx_bar - x_bar*x_bar)
        w0 = y_bar - w1*x_bar
    return w1, w0

w1, w0 = gradient_descent_steps(X, y, 1000)
print(w1, w0)

In [ ]:
y_pred2 = w1 * X + w0
plt.scatter(X, y)
plt.plot(X, y_pred2, c='red')

In [ ]:
# 경사하강법 - 행렬 풀이
X = df.bmi
y = y_log

def get_cost(y, y_pred):
    N = len(y)
    cost = np.sum(np.square(y - y_pred)) / N #손실 함수 or 비용 함수
    return cost

def get_weight_updates(w1, w0, X, y, learning_rate=0.001):
    N = len(y)
    y_pred = np.dot(X, w1) + w0
    print(get_cost(y, y_pred))
    diff = y_pred - y
    ones = np.ones((N, 1))
    w1_update = learning_rate * 2 * np.dot(X.T, diff) / N
    w0_update = learning_rate * 2 * np.dot(ones.T, diff) / N
    return w1_update, w0_update

def gradient_descent_steps(X, y, iters=10000):
    w0 = 0
    w1 = 0
    for _ in range(iters):
        w1_update, w0_update = get_weight_updates(w1, w0, X ,y)
        w1 = w1 - w1_update
        w0 = w0 - w0_update
    return w1, w0

w1, w0 = gradient_descent_steps(X, y, 1000)
print(w1, w0)

In [ ]:
y_pred3 = w1 * X + w0
plt.scatter(X, y)
plt.plot(X, y_pred3, c='red')

In [ ]:
# 경사하강법 - 편미분 방정식 풀이
X = df.bmi
y = y_log

def get_cost(y, y_pred):
    N = len(y)
    cost = np.sum(np.square(y-y_pred))/N
    return cost

def get_weight_updates(w1, w0, X, y):
    N = len(y)
    y_pred =np.sum(X*y)+w0
    print(get_cost(y, y_pred))
    xy_bar = np.sum(X*y)/N
    y_bar = np.sum(y)/N
    x_bar = np.sum(X)/N
    xx_bar =np.sum(X*X)/N
    return xy_bar, y_bar ,x_bar, xx_bar 

def gradient_descent_steps(X, y, iters=10000):
    w0 = 0
    w1 = 0
    for _ in range(iters):
        xy_bar, y_bar, x_bar, xx_bar = get_weight_updates(w1, w0, X, y)
        w1 =(xy_bar - x_bar*y_bar) / (xx_bar - x_bar*x_bar)
        w0 = y_bar - w1*x_bar
    return w1, w0

w1, w0 = gradient_descent_steps(X, y, 1000)
print(w1, w0)

In [ ]:
y_pred4 = w1 * X + w0
plt.scatter(X, y)
plt.plot(X, y_pred4, c='red')

#### 사이킷런 - LinearRegression 알고리즘

In [ ]:
y = y_log
X = df[['age']]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

lr = LinearRegression()
lr.fit(X_train, y_train)
y_hat = lr.predict(X_test)
mae = mean_absolute_error(y_test, y_hat)
mse = mean_squared_error(y_test, y_hat)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_hat)
print(mae, mse, rmse, r2)

In [ ]:
print(lr.intercept_)
print(lr.coef_)

In [ ]:
plt.figure(figsize=(16,8))
plt.scatter(df['age'], y_log, color='black')
plt.plot(X_test, y_hat, c='red', linewidth=2)
plt.show()

In [ ]:
y = y_log
X = df[['bmi']]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

lr = LinearRegression()
lr.fit(X_train, y_train)
y_hat = lr.predict(X_test)
mae = mean_absolute_error(y_test, y_hat)
mse = mean_squared_error(y_test, y_hat)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_hat)
print(mae, mse, rmse, r2)

In [ ]:
print(lr.intercept_)
print(lr.coef_)

In [ ]:
plt.figure(figsize=(16,8))
plt.scatter(df['bmi'], y_log, color='black')
plt.plot(X_test, y_hat, c='red', linewidth=2)
plt.show()

### 다중 선형 회귀

In [ ]:
y = y_log
X = df.drop(['charges'], axis = 1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

lr = LinearRegression()
lr.fit(X_train, y_train)
y_hat = lr.predict(X_test)
mae = mean_absolute_error(y_test, y_hat)
mse = mean_squared_error(y_test, y_hat)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_hat)
print(mae, mse, rmse, r2)

In [ ]:
print(lr.intercept_)
print(lr.coef_)

In [ ]:
y = y_log
X = df.drop(['charges','region'], axis = 1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

lr = LinearRegression()
lr.fit(X_train, y_train)
y_hat = lr.predict(X_test)
mae = mean_absolute_error(y_test, y_hat)
mse = mean_squared_error(y_test, y_hat)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_hat)
print(mae, mse, rmse, r2)

In [ ]:
print(lr.intercept_)
print(lr.coef_)

### 다항 회귀 Polynomial Regression

In [ ]:
y = y_log
X = df1.drop(['charges', 'region'], axis = 1)
X_train, X_test, y_train, y_test = train_test_split(x_pol, y, test_size=0.2)

pol = PolynomialFeatures (degree = 2)
x_pol = pol.fit_transform(X)
Pol_reg = LinearRegression()
Pol_reg.fit(X_train, y_train)
y_test_pred = Pol_reg.predict(X_test)
mae = mean_absolute_error(y_test, y_test_pred)
mse = mean_squared_error(y_test, y_test_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_test_pred)
print(mae, mse, rmse, r2)

In [ ]:
print(Pol_reg.intercept_)
print(Pol_reg.coef_)

### 릿지 회귀

In [ ]:
y = y_log
X = df.drop(['charges','region'], axis = 1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [ ]:
parameters = {'alpha': [0.1, 1, 10, 20, 100, 1000]}
for alpha in parameters:
    ridge = Ridge(alpha=alpha)
    ridgeCV = GridSearchCV(ridge, parameters, 
                           scoring='neg_mean_squared_error', cv=5)
    ridgeCV.fit(X_train, y_train)
    print(ridgeCV.best_params_)
    ridgeCV_pred = ridgeCV.predict(X_test)
    mae = mean_absolute_error(y_test, ridgeCV_pred)
    mse = mean_squared_error(y_test, ridgeCV_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_test, ridgeCV_pred)
    print(mae, mse, rmse, r2)

In [ ]:
parameters = {'alpha': [10, 15, 20, 25, 30, 50, 100, 1000]}
for alpha in parameters:
    ridge = Ridge(alpha=alpha)
    ridgeCV = GridSearchCV(ridge, parameters, 
                           scoring='neg_mean_squared_error', cv=5)
    ridgeCV.fit(X_train, y_train)
    print(ridgeCV.best_params_)
    ridgeCV_pred = ridgeCV.predict(X_test)
    mae = mean_absolute_error(y_test, ridgeCV_pred)
    mse = mean_squared_error(y_test, ridgeCV_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_test, ridgeCV_pred)
    print(mae, mse, rmse, r2)

In [ ]:
parameters = {'alpha': [15, 16, 17, 20, 25, 50]}
for alpha in parameters:
    ridge = Ridge(alpha=alpha)
    ridgeCV = GridSearchCV(ridge, parameters, 
                           scoring='neg_mean_squared_error', cv=5)
    ridgeCV.fit(X_train, y_train)
    print(ridgeCV.best_params_)
    ridgeCV_pred = ridgeCV.predict(X_test)
    mae = mean_absolute_error(y_test, ridgeCV_pred)
    mse = mean_squared_error(y_test, ridgeCV_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_test, ridgeCV_pred)
    print(mae, mse, rmse, r2)

### 라쏘 회귀

In [ ]:
parameters = {'alpha': [1e-3, 1e-2, 1, 5, 10, 20]}
lasso = Lasso()
lassoCV = GridSearchCV(lasso, parameters,
                       scoring='neg_mean_squared_error', cv=5)
lassoCV.fit(X_train, y_train)
print(lassoCV.best_params_)
lassoCV_pred = lassoCV.predict(X_test)
mae = mean_absolute_error(y_test, lassoCV_pred)
mse = mean_squared_error(y_test, lassoCV_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, lassoCV_pred)
print(mae, mse, rmse, r2)

In [ ]:
parameters = {'alpha': [0.001, 0.0012, 0.0013, 0.0014, 0.0015]}
lasso = Lasso()
lassoCV = GridSearchCV(lasso, parameters,
                       scoring='neg_mean_squared_error', cv=5)
lassoCV.fit(X_train, y_train)
print(lassoCV.best_params_)
lassoCV_pred = lassoCV.predict(X_test)
mae = mean_absolute_error(y_test, lassoCV_pred)
mse = mean_squared_error(y_test, lassoCV_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, lassoCV_pred)
print(mae, mse, rmse, r2)

### 엘라스틱넷 회귀

In [ ]:
df1 = df
df.head()

In [ ]:
y = y_log
X = df1.drop(['charges','region'], axis = 1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [ ]:
alphas = [0.07, 0.1, 0.5, 1, 3]
for alpha in alphas:
    enet_model = ElasticNet(alpha=alpha, l1_ratio=0.7).fit(X_train,y_train)
    enetYpred_ = enet_model.predict(X_test)
    mae = mean_absolute_error(y_test, enetYpred_)
    mse = mean_squared_error(y_test, enetYpred_)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_test, enetYpred_)
    print(alpha, max(mae, mse, rmse, r2))

In [ ]:
alphas = [0.07, 0.08, 0.09, 0.1, 0.12]
for alpha in alphas:
    enet_model = ElasticNet(alpha=alpha, l1_ratio=0.7).fit(X_train,y_train)
    enetYpred_ = enet_model.predict(X_test)
    mae = mean_absolute_error(y_test, enetYpred_)
    mse = mean_squared_error(y_test, enetYpred_)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_test, enetYpred_)
    print(alpha, max(mae, mse, rmse, r2))

In [ ]:
enet_model = ElasticNet(alpha=0.07, l1_ratio=0.7).fit(X_train,y_train)
enetYpred_ = enet_model.predict(X_test)
mae = mean_absolute_error(y_test, enetYpred_)
mse = mean_squared_error(y_test, enetYpred_)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, enetYpred_)
print(alpha, mae, mse, rmse, r2)

### 로지스틱 회귀

In [ ]:
df2 = df
df2.describe()

In [ ]:
df2.loc[df1['charges'] <= 8434, 'charges'] = 0 #on or under avarage
df2.loc[df1['charges'] > 8434, 'charges'] = 1 # above avarage
df2.describe()

In [ ]:
df2.groupby(['charges']).count()

In [ ]:
sns.catplot(x="charges", kind="count", palette="cool", data=df2)

In [ ]:
feature_cols = ['age', 'sex', 'bmi', 'children'] 
X = df1[feature_cols] 
y = df1['charges'] 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

lg = LogisticRegression()
lg.fit(X_train, y_train)
lg.score(X_test, y_test)

### 랜덤 포레스트 ( Random Forest ) Regressor

In [ ]:
df1.head()

In [ ]:
X = df1.drop(['charges', 'region'], axis=1)
y = df1.charges
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

Rfr = RandomForestRegressor(n_estimators = 100, criterion = 'mse',
                              random_state = 1,
                              n_jobs = -1)
Rfr.fit(X_train,y_train)
X_train_pred = Rfr.predict(X_train)
X_test_pred = Rfr.predict(X_test)

mae = mean_absolute_error(y_test, X_test_pred)
mse = mean_squared_error(y_test, X_test_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, X_test_pred)
print(mae, mse, rmse, r2)

In [ ]:
plt.figure(figsize=(8,6))

plt.scatter(X_train_pred, X_train_pred - y_train,
          c = 'gray', marker = 'o', s = 35, alpha = 0.5,
          label = 'Train data')
plt.scatter(X_test_pred, X_test_pred - y_test,
          c = 'blue', marker = 'o', s = 35, alpha = 0.7,
          label = 'Test data')
plt.xlabel('Predicted values')
plt.ylabel('Actual values')
plt.legend(loc = 'upper right')
plt.hlines(y = 0, xmin = 0, xmax = 30000, lw = 2, color = 'red')

In [ ]:
print('Feature importance ranking')
importances = Rfr.feature_importances_
std = np.std([tree.feature_importances_ for tree in Rfr.estimators_],axis=0)
indices = np.argsort(importances)[::-1]
variables = ['age', 'sex', 'bmi', 'children']
importance_list = []
for f in range(X.shape[1]):
    variable = variables[indices[f]]
    importance_list.append(variable)
    print("%d.%s(%f)" % (f + 1, variable, importances[indices[f]]))

In [ ]:
# Plot the feature importances of the forest
plt.figure()
plt.title("Feature importances")
plt.bar(importance_list, importances[indices],
       color="y", yerr=std[indices], align="center")

### 회귀 트리

#### StandardScaler

In [ ]:
df1.head()

In [ ]:
y = y_log
X = df1.drop(['charges','region'], axis = 1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.fit_transform(X_test)

In [ ]:
lr = LinearRegression()

knn = KNeighborsRegressor(n_neighbors=10)

dt = DecisionTreeRegressor(max_depth = 3)

rf = RandomForestRegressor(max_depth = 3, n_estimators=500)

ada = AdaBoostRegressor( n_estimators=50, learning_rate =.01)

gbr = GradientBoostingRegressor(max_depth=2, n_estimators=100, learning_rate =.2)

xgb = XGBRegressor(max_depth = 3, n_estimators=50, learning_rate =.2)

lgb = LGBMRegressor(max_depth = 3, n_estimators=500)

regressors = [('Linear Regression', lr), ('K Nearest Neighbours', knn),
               ('Decision Tree', dt), ('Random Forest', rf), ('AdaBoost', ada),
              ('Gradient Boosting Regressor', gbr), ('XGBoost', xgb),('LGBM Regressor', lgb)]

In [ ]:
for regressor_name, regressor in regressors:
 
    # Fit regressor to the training set
    regressor.fit(X_train, y_train)    
   
    # Predict 
    y_pred = regressor.predict(X_test)
    accuracy = round(r2_score(y_test,y_pred),1)*100
   
    # Evaluate  accuracy on the test set
    print('{:s} : {:.0f} %'.format(regressor_name, accuracy))
    plt.rcParams["figure.figsize"] = (12,8)
    plt.bar(regressor_name,accuracy)

#### MinMaxScaler

In [ ]:
df1.head()

In [ ]:
y = y_log
X = df1.drop(['charges','region'], axis = 1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

sc = MinMaxScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.fit_transform(X_test)

In [ ]:
lr = LinearRegression()

knn = KNeighborsRegressor(n_neighbors=10)

dt = DecisionTreeRegressor(max_depth = 3)

rf = RandomForestRegressor(max_depth = 3, n_estimators=500)

ada = AdaBoostRegressor( n_estimators=50, learning_rate =.01)

gbr = GradientBoostingRegressor(max_depth=2, n_estimators=100, learning_rate =.2)

xgb = XGBRegressor(max_depth = 3, n_estimators=50, learning_rate =.2)

lgb = LGBMRegressor(n_estimators=1000)

regressors = [('Linear Regression', lr), ('K Nearest Neighbours', knn),
               ('Decision Tree', dt), ('Random Forest', rf), ('AdaBoost', ada),
              ('Gradient Boosting Regressor', gbr), ('XGBoost', xgb),('LGBM Regressor', lgb)]

In [ ]:
for regressor_name, regressor in regressors:
 
    # Fit regressor to the training set
    regressor.fit(X_train, y_train)    
   
    # Predict 
    y_pred = regressor.predict(X_test)
    accuracy = round(r2_score(y_test,y_pred),1)*100
   
    # Evaluate  accuracy on the test set
    print('{:s} : {:.0f} %'.format(regressor_name, accuracy))
    plt.rcParams["figure.figsize"] = (12,8)
    plt.bar(regressor_name,accuracy)